In [ ]:
import opengrid as og

In [ ]:
og.datasets.list_available()

In [ ]:
%matplotlib inline

In [ ]:
ds = og.datasets.get('elec_power_min_1sensor')
ds.head()

In [ ]:
# we only want to demo on one sensor
ts = ds.head(300)

In [ ]:
ts.plot()

### We want to know the minimum elektricity power during the series

In [ ]:
ts.min()